In [9]:
!pip install weaviate-client

  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ---------------------------------------- 0.0/325.7 kB ? eta -:--:--
   -------------------------------------- - 317.4/325.7 kB 9.9 MB/s eta 0:00:01
   ---------------------------------------- 325.7/325.7 kB 6.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/40.0 kB ? eta -:--:--
   ---------------------------------------- 40.0/40.0 kB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/223.8 kB ? eta -:--:--
   --------------------------------------- 223.8/223.8 kB 13.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------  1.1/1.1 MB 34.4 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 13.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/426.9 kB ? eta -:--:--
   --------------------------------------- 426.9/426.9 kB 26.0 MB/s eta 0:00:00
   -------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 2.19.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.27.1 which is incompatible.
google-cloud-aiplatform 1.56.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.1 which is incompatible.
google-cloud-resource-manager 1.12.3 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.1 which is incompatible.
grpc-google-iam-v1 0.13.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.1 which is incompatible.
opentelemetry-proto 1.25.0 requires protobuf<5.0,>=3.19, but you have p

In [40]:
!pip install cohere

c:\Users\Admin\.conda\envs\interview-prep\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedWriter name=3>
  return process_handler(cmd, _system_body)
c:\Users\Admin\.conda\envs\interview-prep\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=4>
  return process_handler(cmd, _system_body)
c:\Users\Admin\.conda\envs\interview-prep\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)


In [1]:
from langchain.document_loaders import PyPDFLoader, CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever, ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain.retrievers.weaviate_hybrid_search import WeaviateHybridSearchRetriever
from langchain.chat_models.openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import weaviate
import os

In [42]:
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
WEAVIATE_API_KEY = os.getenv('WEAVIATE_API_KEY')
WEAVIATE_CLUSTER_ENV = os.getenv('WEAVIATE_CLUSTER_ENV')
COHERE_API_KEY = os.getenv('COHERE_API_KEY')

In [2]:
loader1 = CSVLoader("./data/final_data.csv")
document1 = loader1.load()

In [5]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text_chunks = splitter.split_documents(document1)

In [5]:
embeddings = OpenAIEmbeddings()

c:\Users\Admin\.conda\envs\interview-prep\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [36]:
PROMPT_TEMPLATE = ("""
Use the following pieces of information to answer the user's question.
If you dont know the answer just say you don't know, don't try to make up an answer.

Context:{context}

Only return the helpful answer below and nothing else
Helpful answer
Question: {question}                   
"""
)

In [37]:
prompt_template = PromptTemplate(input_variables=["context", "question"], template=PROMPT_TEMPLATE)

In [14]:
client = weaviate.Client(url=WEAVIATE_CLUSTER_ENV, auth_client_secret= weaviate.auth.AuthApiKey(WEAVIATE_API_KEY), 
                         additional_headers={"X-OpenAI-Api-Key": OPENAI_API_KEY},)

c:\Users\Admin\.conda\envs\interview-prep\lib\site-packages\weaviate\warnings.py:162: DeprecationWarning: Dep016: Python client v3 `weaviate.Client(...)` connections and methods are deprecated. Update
            your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.

            For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
            For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration
            
  warnings.warn(


In [15]:
client.is_ready()

True

In [16]:
client.schema.get()

{'classes': []}

In [20]:
client.schema.get()

{'classes': [{'class': 'SAG1',
   'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
    'cleanupIntervalSeconds': 60,
    'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
   'moduleConfig': {'text2vec-openai': {'baseURL': 'https://api.openai.com',
     'model': 'ada',
     'modelVersion': '002',
     'type': 'text',
     'vectorizeClassName': True}},
   'multiTenancyConfig': {'autoTenantActivation': False,
    'autoTenantCreation': False,
    'enabled': False},
   'properties': [{'dataType': ['text'],
     'indexFilterable': True,
     'indexSearchable': True,
     'moduleConfig': {'text2vec-openai': {'skip': False,
       'vectorizePropertyName': True}},
     'name': 'content',
     'tokenization': 'word'},
    {'dataType': ['text'],
     'indexFilterable': True,
     'indexSearchable': True,
     'moduleConfig': {'text2vec-openai': {'skip': False,
       'vectorizePropertyName': False}},
     'name': 'source',
     'tokenization': 'word'},
    {'dataType': [

In [22]:
retriever = WeaviateHybridSearchRetriever(client=client, 
                                          index_name="SAG1", 
                                          alpha=0.5, #param to balance b/w keyword and semantic search
                                          text_key="content", 
                                          attributes=[], 
                                          create_schema_if_missing=True)

In [23]:
retriever.add_documents(text_chunks)

['43a6c19f-84cc-4477-986a-b709ada5522e',
 '4470fc94-acd9-42f2-8acd-669eeccb70e4',
 '30d35753-19e0-45a4-9433-0607cd4ca5ec',
 '73d8ffbc-31ce-43aa-834c-6e5457f6d019',
 '2af7a8f8-6032-41b3-bc68-c7e58fd705b7',
 '8327b890-4458-49d9-ab7e-a14de3593282',
 '613ee495-a3c9-4644-bdb4-199c70351acb',
 '9baef35d-66b6-47f0-8ed2-285a9efb6729',
 'dff79c76-b5c1-479c-b3f5-c3c316e4e377',
 'a8ebf901-57a9-418f-81a9-dee997ec74ec',
 'f7bf6200-a46c-4a35-9db8-97da3e9b8bed',
 '95326e29-976c-43c6-80aa-ed82f61f3136',
 '56e87b04-7ad2-448d-bac1-a886429433f9',
 '0b3390ed-df51-4450-9a79-265f6af6607b',
 '2e4ec107-faee-4114-b303-3c6ca0c797e7',
 '410a7d3c-1520-4554-bc8b-b8a3c1d42c6e',
 '5b3fb08b-8386-41d3-abd8-0854faca8509',
 'cb980819-b28b-4837-afd2-0093fe8d27b0',
 '210d16bc-58b0-4fa5-b33b-a68f53559483',
 '67932173-9c38-4b18-a984-fca4e0e67c21',
 'e744b03b-0778-4942-be0d-f74e0c32bb46',
 '71639387-0f26-48ef-b34f-8f5d5c00fe80',
 '20f9646c-7768-459d-b715-9d171c669656',
 '628bfbbe-4441-4f85-bd15-fee22f8ec15a',
 '34d2a1d7-8e14-

In [24]:
retriever.invoke("How to make Requested Start Date and Requested End Date mandatory in CR creation?", score=True)

[Document(page_content='subject: Configuration for Requested Dates Mandatory during Change Create.\ncontent: As part of the current CM B2B design “Requested Start Date” and “Requested End Date” is mandatory to create the CR irrespective of any change class and customer system need to send these two date/time values while sending the Cerate request. As per the OFR requirement these two date/time fields should not be mandatory to create the CR. So following changes need to be done in the existing design to fulfill the requirement.\nIn the form “Eric:CustomerInterface” 5 checkboxes has been created as. Refer below screen shot.\nNormal\nEmergency\nExpedited\nNo Impact\nStandard\nFor any customer if Requested Start date and End date need to be set as “Mandatory” for a particular change Class then the respective check box need to be checked. By Default For CR Type Latent Requested Start Data and End Dates are mandatory.', metadata={'_additional': {'explainScore': '\nHybrid (Result Set keywor

In [4]:
llm = ChatOpenAI()

c:\Users\Admin\.conda\envs\interview-prep\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [54]:
retrieval_chain = ({"context": retriever, "question": RunnablePassthrough()} |
                   prompt_template |
                   llm |
                   StrOutputParser())

In [55]:
response = retrieval_chain.invoke("How to make Requested Start Date and Requested End Date mandatory in CR?")
response

c:\Users\Admin\.conda\envs\interview-prep\lib\site-packages\pydantic\main.py:1070: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


'Based on the information provided, it appears that in the existing CM B2B design, the "Requested Start Date" and "Requested End Date" fields are already mandatory to create a Change Request (CR). By default, for CR Type Latent, these dates are mandatory. Additionally, checkboxes can be used to set these fields as mandatory for a particular change class.'

In [43]:
cohere = CohereRerank(cohere_api_key=COHERE_API_KEY)

c:\Users\Admin\.conda\envs\interview-prep\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `CohereRerank` was deprecated in LangChain 0.0.30 and will be removed in 0.3.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import CohereRerank`.
  warn_deprecated(


In [44]:
compress_retriever = ContextualCompressionRetriever(base_compressor=cohere, base_retriever=retriever)

In [67]:
retrieval_chains = ({"context": compress_retriever, "question": RunnablePassthrough()} |
                   prompt_template |
                   llm |
                   StrOutputParser())

In [59]:
type(retrieval_chains)

langchain_core.runnables.base.RunnableSequence

In [69]:
resposne1 = retrieval_chains.invoke("How to configure support groups in ITSM")
resposne1

c:\Users\Admin\.conda\envs\interview-prep\lib\site-packages\pydantic\main.py:1070: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


'To configure support groups in ITSM, you need to follow these steps:\n1. Login to ITSM NG and Open People form.\n2. Open People profile with Login ID as NetworkActuation.\n3. Go to Support Groups Tab.\n4. Open Support Group and add newly created support group.\n5. Open Support Group Function Role tab and add newly created group, giving functional role as Infrastructure Change Coordinator.'

In [70]:
resposne2 = retrieval_chain.invoke("How to configure support groups in ITSM")
resposne2

c:\Users\Admin\.conda\envs\interview-prep\lib\site-packages\pydantic\main.py:1070: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


'To configure support groups in ITSM, follow these steps:\n\n1. On the Standard Configuration tab of the Application Administration Console, select the correct company.\n2. Click the Create link next to Support Group.\n3. Enter or select information in the Support Organization field.\n4. Enter the name of the support organization.\n5. Enter or select information in the Support Group Name field.\n6. Enter the support group role (e.g. Help Desk, Tier 1, Tier 2, Line of Business).\n7. Click Add to add the support group.\n8. Repeat the process to add all required support groups.\n9. When finished, click Close to save the configuration.'

In [72]:
resposne1 = retrieval_chains.invoke("How to make Requested Start Date and Requested End Date mandatory in CR?")
resposne1

c:\Users\Admin\.conda\envs\interview-prep\lib\site-packages\pydantic\main.py:1070: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


'To make Requested Start Date and Requested End Date mandatory in CR, you need to check the respective checkboxes in the form "Eric:CustomerInterface" for the specific change class. This action will set the Requested Start Date and End Date as mandatory for creating a CR.'

In [76]:
resposne3 = retrieval_chain.invoke("which all modules of ITSM are supported for ITSM-BPMS interface?")
resposne3

c:\Users\Admin\.conda\envs\interview-prep\lib\site-packages\pydantic\main.py:1070: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


'Based on the information provided, the ITSM-BPMS interface supports modules such as Change Management, Problem Management, and Incident Management within ITSM.'

In [ ]:
retrieval_chain.ainvoke